<a href="https://colab.research.google.com/github/neethusunil/100daysofcodepython/blob/master/notebook/Custom_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import re
import math
import numpy as np
from scipy.spatial.distance import euclidean
from difflib import SequenceMatcher
import numpy as np
from numpy.linalg import norm
from scipy import spatial
from math import *
from decimal import Decimal
import nltk
from nltk.translate.bleu_score import sentence_bleu

In [12]:
actual = "startseq 1x接種x0 91xCUTx0 96xHx4520 97xHx94660 98xHx37660 99xHx14400 100xZx1000 105xWx2660 106xWMx0 111xHx25460 112xHx27820 113xHx14440 115xHx3660 116xZx0 118xWx0 123xHx11460 124xHx43680 125xHx46180 126xHx15040 127xHx7280 128xHx560 130xZx0 132xHx30 133xWx0 137xHx1120 138xHx19840 139xHx30380 140xHx16740 141xHx5780 142xHx760 148x廃棄x40 endseq"
predicted = "startseq 1x接種x0 91xCUTx0 96xHx4520 97xHx94660 98xHx37660 99xHx14400 100xZx1000 105xWx2660 106xWMx0 111xHx25460 112xHx27820 113xHx14440 115xHx3660 116xZx0 118xWx0 123xHx11460 124xHx43680 125xHx46180 126xHx15040 133xHx1940 131xWx0 132xWMx0 142xHx10800 130xHx3760 136xHx9340 134xHx9600 147x廃棄x1320 endseq"

In [13]:
def find_yield(seq):
    seq.split() #split the sequence based on space
    pattern = "(?<=x)\d+" #regex expression
    sp = re.findall(pattern, seq) #print all the yield values
    sp = [int(i) for i in sp] #converted string to array
    return sp

def find_day(seq):
  seq.split()
  pattern = "(\d+)x"
  sp = re.findall(pattern,seq)
  sp = [int(i) for i in sp]
  return sp


def day_count(act,pred):
    print(act)
    x=act.split()
    y=pred.split()
    def day_no(z):
        if '廃棄' in z[-2]: #end day work or waste
            end_day = re.findall(r"(\d+)x", z[-2])
            end_day = int(end_day[0])
            return end_day
    
    act = day_no(x)
    pred = day_no(y)
    
    days = {"Actual": act,
          "Predicted": pred,
          "Difference": abs(act - pred)
          }
    return days

def work_count(act,pred):
  act = act.split()
  pred = pred.split()
  act = len(act) - 2
  pred = len(pred) -2
  count =  {"Actual": act,
          "Predicted": pred,
          "Difference": abs(act - pred)
          }

  return count

def work_day(seq):
  pattern = "(?<=x)\d+"
  ans=re.sub(pattern,'',seq)
  ans=ans.split()
  ans.remove('startseq')
  ans.remove('endseq')
  ans = [x[:-1] for x in ans] #remove 'x' in the end of all the elements
  return ans

def work_yield(seq):
  
  seq.split()
  pattern = '[^x]*x\d+'#
  sp=re.findall(pattern,seq)
  return sp


def similar(a,b):                      #similarity in nlp using sequence matcher
  return SequenceMatcher(None, a, b).ratio()

# Function distance between two points
# and calculate distance value to given
# root value(p is root value)
def p_root(value, root):
  root_value = 1 / float(root)
  val = round (Decimal(value) ** Decimal(root_value), 3)
  return val

def minkowski_distance(x, y, p_value):
  # pass the p_root function to calculate
  # all the value of vector parallelly
  return (p_root(sum(pow(abs(a-b), p_value)
      for a, b in zip(x, y)), p_value))
    
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)




In [14]:
def custom_eval():
    inp = find_yield(actual)
    out = find_yield(predicted)
    day_in = find_day(actual)
    day_out = find_day(predicted)
    p=3
    summary = { 
        # "Actual Yield Values":inp,
        # "Predicted Yield Values":out,
        "Number of Days": day_count(actual,predicted),
        "Work Entries": work_count(actual,predicted),
        "Difference in total yield Value":abs(sum(inp)-sum(out)),
        "Similarity Score:":similar(actual,predicted),
        "Jaccard Similarity ": jaccard_similarity(actual, predicted),
        "Jaccard Similarity for yield values": jaccard_similarity(inp, out),
        "Jaccard Similarity for day values": jaccard_similarity(day_in, day_out),
        "Minkowski b/w Yield Values ":minkowski_distance(inp, out, p),        
        "Minkowski b/w Day Values ":minkowski_distance(day_in, day_out, p),
        "Sentence Bleu Score Work x Day": sentence_bleu([work_day(actual)],work_day(predicted)),
        "Sentence Bleu Score Work x Yield": sentence_bleu([work_yield(actual)],work_yield(predicted))   
        
        
    }
    return(summary)

In [15]:
custom_eval()

startseq 1x接種x0 91xCUTx0 96xHx4520 97xHx94660 98xHx37660 99xHx14400 100xZx1000 105xWx2660 106xWMx0 111xHx25460 112xHx27820 113xHx14440 115xHx3660 116xZx0 118xWx0 123xHx11460 124xHx43680 125xHx46180 126xHx15040 127xHx7280 128xHx560 130xZx0 132xHx30 133xWx0 137xHx1120 138xHx19840 139xHx30380 140xHx16740 141xHx5780 142xHx760 148x廃棄x40 endseq


{'Number of Days': {'Actual': 148, 'Predicted': 147, 'Difference': 1},
 'Work Entries': {'Actual': 31, 'Predicted': 27, 'Difference': 4},
 'Difference in total yield Value': 45770,
 'Similarity Score:': 0.6957878315132605,
 'Jaccard Similarity ': 1.0,
 'Jaccard Similarity for yield values': 0.4838709677419355,
 'Jaccard Similarity for day values': 0.6571428571428571,
 'Minkowski b/w Yield Values ': Decimal('30229.608'),
 'Minkowski b/w Day Values ': Decimal('12.287'),
 'Sentence Bleu Score Work x Day': 0.5987642155325291,
 'Sentence Bleu Score Work x Yield': 0.5987642155325291}